In [1]:
import time, sys
import web3 as w3; w3 = w3.Account
from objsize import get_deep_size as get_size
from eth_account.messages import encode_defunct as encode_msg
from hashlib import sha256

In [2]:
# print hash
ph = lambda h: h[:16] + '...' + h[-3:]

# random hash
rh = lambda: sha256(str(time.time()).encode()).hexdigest()

In [3]:
class Account:
    def __init__(self): 
        self.priv, self.pub = Account.keys()
        self.nonce = 0
        
    @staticmethod
    def keys():
        acc = w3.create()
        return acc.privateKey.hex(), acc.address
    
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        return w3.sign_message(m, self.priv)
    
    def verify(self, tx, sig):
        m = encode_msg(bytes(tx))
        return w3.recover_message(m, signature=sig.signature) == self.pub
    
    def mine(self, block, diff, reward, attempts=1000):
        mblock   = MinebleBlock(*unpack(block), self.pub, diff, reward, block.size())
        mblock_b = bytes(mblock)
        nonce = 0
        for i in range(attempts):
            candidate   = mblock_b + str(nonce).encode()
            candidate_h = sha256(candidate).hexdigest()
            if candidate_h[:diff] == '0'*diff: return MinedBlock(*unpack(mblock), nonce)
            nonce += 1

In [4]:
acc1 = Account()
acc2 = Account()

ph(acc1.pub), ph(acc2.pub)

('0xAe034382ff57aC...776', '0x0efC172a4688E7...5d4')

In [6]:
class TX: 
    def __init__(self, fr, to, value, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.nonce       = nonce
        self.time        = time.time()
    def __str__ (self): return ('time:\t' + time.ctime(self.time)+
                                '\nfrom:\t'   + ph(self.fr)+
                                '\nto:\t'   + ph(self.to)+
                                '\nvalue:\t'+ str(self.value)+' ether' 
                                '\nnonce:\t'+ str(self.nonce))
    def __bytes__(self): return (self.fr+
                                 self.to+
                                 str(self.value)+
                                 str(self.nonce)+
                                 str(self.time)).encode()
    
tx1 = TX(acc1.pub, acc2.pub, 3, acc1.nonce)
tx2 = TX(acc2.pub, acc1.pub, 9, acc2.nonce)

print(tx1, '\n')
print(tx2)

time:	Wed Mar 24 23:56:27 2021
from:	0xAe034382ff57aC...776
to:	0x0efC172a4688E7...5d4
value:	3 ether
nonce:	0 

time:	Wed Mar 24 23:56:27 2021
from:	0x0efC172a4688E7...5d4
to:	0xAe034382ff57aC...776
value:	9 ether
nonce:	0


In [7]:
sig1 = acc1.sign(tx1)
assert acc1.verify(tx1, sig1)

sig2 = acc2.sign(tx2)
assert acc2.verify(tx2, sig2)

In [8]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
def unpack(o): return o.__dict__.values()

In [9]:
class Block: 
    def __init__(self, txs, prev_hash, number, time):
        self.txs       = txs
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time
    def size(self): return get_size(self)
    def __str__(self): return ('time:\t\t'        + time.ctime(self.time)+
                               '\nnumber:\t\t'    + str(self.number)+
                               '\nprev_hash:\t'   + ph(self.prev_hash)+
                               '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()
        
block = Block([tx1, tx2], rh(), 2, time.time())
print(block)

time:		Wed Mar 24 23:56:35 2021
number:		2
prev_hash:	8c2bceeaee2bff09...890

txs:		

time:	Wed Mar 24 23:56:27 2021
from:	0xAe034382ff57aC...776
to:	0x0efC172a4688E7...5d4
value:	3 ether
nonce:	0

time:	Wed Mar 24 23:56:27 2021
from:	0x0efC172a4688E7...5d4
to:	0xAe034382ff57aC...776
value:	9 ether
nonce:	0



In [10]:
class MinebleBlock(Block):
    def __init__(self, 
                 txs, 
                 prev_hash, 
                 number, 
                 time, 
                 miner, 
                 diff, 
                 reward, 
                 size):
        super().__init__(txs, prev_hash, number, time)
        self.miner  = miner
        self.diff   = diff
        self.reward = reward
        self.size   = size
    def __str__(self): return (super().__str__() + 
                               '\nminer:\t\t'    + ph(self.miner)+
                               '\ndiff:\t\t'     + str(self.diff)+
                               '\nreward:\t\t'   + str(self.reward)+' ether'+
                               '\nsize:\t\t'     + str(self.size)+' bytes')
    def __bytes__(self): return (super().__bytes__().decode()+
                                 self.miner+
                                 str(self.diff)+
                                 str(self.reward)+
                                 str(self.size)).encode()
    
mblock = MinebleBlock(*unpack(block), rh(), 3, 100, block.size())
print(mblock)

time:		Wed Mar 24 23:56:35 2021
number:		2
prev_hash:	8c2bceeaee2bff09...890

txs:		

time:	Wed Mar 24 23:56:27 2021
from:	0xAe034382ff57aC...776
to:	0x0efC172a4688E7...5d4
value:	3 ether
nonce:	0

time:	Wed Mar 24 23:56:27 2021
from:	0x0efC172a4688E7...5d4
to:	0xAe034382ff57aC...776
value:	9 ether
nonce:	0

miner:		ed9005b2a70516d9...aef
diff:		3
reward:		100 ether
size:		1003 bytes


In [11]:
class MinedBlock(MinebleBlock):
    def __init__(self, 
                 txs, 
                 prev_hash, 
                 number, 
                 time, 
                 miner, 
                 diff, 
                 reward, 
                 size, 
                 nonce):
        super().__init__(txs, prev_hash, number, time, miner, diff, reward, size)
        self.nonce         = nonce
        self.hash          = sha256(bytes(self)).hexdigest() 
    
    def verify_pow(self):
        candidate   = super().__bytes__()+str(self.nonce).encode()
        candidate_h = sha256(candidate).hexdigest()
        return candidate_h[:self.diff] == '0'*self.diff
    
    def __str__(self):   return (super().__str__() +
                               '\nnonce:\t\t'      + str(self.nonce)+
                               '\nhash:\t\t'       + ph(self.hash))
    def __bytes__(self): return (super().__bytes__().decode() + str(self.nonce)).encode()
    
mined_block = MinedBlock(*unpack(mblock), 70)
print(mined_block)

time:		Wed Mar 24 23:56:35 2021
number:		2
prev_hash:	8c2bceeaee2bff09...890

txs:		

time:	Wed Mar 24 23:56:27 2021
from:	0xAe034382ff57aC...776
to:	0x0efC172a4688E7...5d4
value:	3 ether
nonce:	0

time:	Wed Mar 24 23:56:27 2021
from:	0x0efC172a4688E7...5d4
to:	0xAe034382ff57aC...776
value:	9 ether
nonce:	0

miner:		ed9005b2a70516d9...aef
diff:		3
reward:		100 ether
size:		1003 bytes
nonce:		70
hash:		bdffcb6d5a2d3407...dfe


In [17]:
mined_block = acc1.mine(block,3,100,10000)
assert mined_block.verify_pow()

In [64]:
class Blockchain:
    def __init__(): pass